<a href="https://colab.research.google.com/github/ssheggrud/Mod_20_Project/blob/05_Riley/ETL_Expenditures_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-2.4.8'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-updates/restricted amd64 Packages [594 kB]
Hit:15 http://ppa

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-09-07 17:50:48--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2021-09-07 17:50:48 (11.3 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark = SparkSession.builder.appName("FinalProject").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://databootcamp-final-05.s3.amazonaws.com/Resources/Expenditures+2021+(UF).csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Expenditures+2021+(UF).csv"), sep=",", header=True)
df.show()

+--------+--------+------+--------+--------+---------+------+---------+------+--------+------+----------+--------+---------+----------+--------------------+------+-------+-----+-------+---------+---------------+-----+-----+----------+--------+---------+----------------+-----------------+--------+------+-------+
|ELECTION|OFFICECD|CANDID|CANCLASS|CANDLAST|CANDFIRST|CANDMI|COMMITTEE|FILING|SCHEDULE|PAGENO|SEQUENCENO|   REFNO| INV_DATE|      DATE|                NAME|C_CODE|ORG_IND|STRNO|STRNAME|APARTMENT|           CITY|STATE|  ZIP|PAY_METHOD|    AMNT|PURPOSECD|         PURPOSE|          EXPLAIN|EXEMPTCD|RR_IND|SEG_IND|
+--------+--------+------+--------+--------+---------+------+---------+------+--------+------+----------+--------+---------+----------+--------------------+------+-------+-----+-------+---------+---------------+-----+-----+----------+--------+---------+----------------+-----------------+--------+------+-------+
|    2021|       1|  2445|      NP| Francis|   Quanda|     S|

In [5]:
df.columns

['ELECTION',
 'OFFICECD',
 'CANDID',
 'CANCLASS',
 'CANDLAST',
 'CANDFIRST',
 'CANDMI',
 'COMMITTEE',
 'FILING',
 'SCHEDULE',
 'PAGENO',
 'SEQUENCENO',
 'REFNO',
 'INV_DATE',
 'DATE',
 'NAME',
 'C_CODE',
 'ORG_IND',
 'STRNO',
 'STRNAME',
 'APARTMENT',
 'CITY',
 'STATE',
 'ZIP',
 'PAY_METHOD',
 'AMNT',
 'PURPOSECD',
 'PURPOSE',
 'EXPLAIN',
 'EXEMPTCD',
 'RR_IND',
 'SEG_IND']

In [6]:
# Remove multiple columns
list = ['CANDMI','COMMITTEE',
 'FILING',
 'SCHEDULE',
 'PAGENO',
 'SEQUENCENO',
 'REFNO',
 'INV_DATE',
 'DATE',
 'NAME',
 'ORG_IND',
 'STRNO',
 'STRNAME',
 'APARTMENT',
  'EXPLAIN',
 'EXEMPTCD',
 'RR_IND',
 'SEG_IND'] 
df = df.drop(*list)
df.show()

+--------+--------+------+--------+--------+---------+------+---------------+-----+-----+----------+--------+---------+----------------+
|ELECTION|OFFICECD|CANDID|CANCLASS|CANDLAST|CANDFIRST|C_CODE|           CITY|STATE|  ZIP|PAY_METHOD|    AMNT|PURPOSECD|         PURPOSE|
+--------+--------+------+--------+--------+---------+------+---------------+-----+-----+----------+--------+---------+----------------+
|    2021|       1|  2445|      NP| Francis|   Quanda|  CORP|       Brooklyn|   NY|11201|      null| 2001.16|    RENTO|     Office Rent|
|    2021|      11|  2546|       P|    Yang|   Andrew|  CORP|       Glendale|   NY|11385|   Unknown|   91.57|    OTHER|Other: explntion|
|    2021|      11|   148|       P|Stringer|    Scott|   LLC|       New York|   NY|10006|   Unknown| 3240.75|    RENTO|     Office Rent|
|    2021|      11|   148|       P|Stringer|    Scott|   LLC|       New York|   NY|10006|   Unknown|13240.75|    RENTO|     Office Rent|
|    2021|      11|   148|       P|String

In [7]:
# check data types
df.dtypes

[('ELECTION', 'string'),
 ('OFFICECD', 'string'),
 ('CANDID', 'string'),
 ('CANCLASS', 'string'),
 ('CANDLAST', 'string'),
 ('CANDFIRST', 'string'),
 ('C_CODE', 'string'),
 ('CITY', 'string'),
 ('STATE', 'string'),
 ('ZIP', 'string'),
 ('PAY_METHOD', 'string'),
 ('AMNT', 'string'),
 ('PURPOSECD', 'string'),
 ('PURPOSE', 'string')]

In [8]:
df.columns

['ELECTION',
 'OFFICECD',
 'CANDID',
 'CANCLASS',
 'CANDLAST',
 'CANDFIRST',
 'C_CODE',
 'CITY',
 'STATE',
 'ZIP',
 'PAY_METHOD',
 'AMNT',
 'PURPOSECD',
 'PURPOSE']

In [9]:
#Change column names
df1 = df \
.withColumnRenamed("ELECTION", "Election") \
.withColumnRenamed("CANDID", "CandidateID") \
.withColumnRenamed("CANDLAST", "CandidateLastName") \
.withColumnRenamed('CANDFIRST', "CandidateFirstName") \
.withColumnRenamed("C_CODE", "ContributionType") \
.withColumnRenamed("CITY", "City") \
.withColumnRenamed("STATE", "State") \
.withColumnRenamed("ZIP", "ZipCode") \
.withColumnRenamed("AMNT", "Amount") \
.withColumnRenamed("PURPOSECD", "PurposeCode") \
.withColumnRenamed("PURPOSE", "Purpose") \
.withColumnRenamed("PAY_METHOD", "PaymentMethod") 

df1.printSchema()

root
 |-- Election: string (nullable = true)
 |-- OFFICECD: string (nullable = true)
 |-- CandidateID: string (nullable = true)
 |-- CANCLASS: string (nullable = true)
 |-- CandidateLastName: string (nullable = true)
 |-- CandidateFirstName: string (nullable = true)
 |-- ContributionType: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ZipCode: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- PurposeCode: string (nullable = true)
 |-- Purpose: string (nullable = true)



In [10]:
# changing datatypes of columns
df2 = df1 \
  .withColumn("Election",df1["Election"].cast("Integer")) \
  .withColumn("OFFICECD", df1["OFFICECD"].cast("Integer")) \
  .withColumn("CandidateID", df1["CandidateID"].cast("Integer")) \
  .withColumn("ZipCode", df1["ZipCode"].cast("Integer")) \
  .withColumn("Amount", df1["Amount"].cast("Float")) \
  
df2.printSchema()

root
 |-- Election: integer (nullable = true)
 |-- OFFICECD: integer (nullable = true)
 |-- CandidateID: integer (nullable = true)
 |-- CANCLASS: string (nullable = true)
 |-- CandidateLastName: string (nullable = true)
 |-- CandidateFirstName: string (nullable = true)
 |-- ContributionType: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ZipCode: integer (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- Amount: float (nullable = true)
 |-- PurposeCode: string (nullable = true)
 |-- Purpose: string (nullable = true)



In [14]:
#Change vaule name in ContributerType
from pyspark.sql.functions import regexp_replace
df3 = df2.withColumn("ContributionType", regexp_replace("ContributionType", 'CAN', 'Candidate')) \
  .withColumn('ContributerType', regexp_replace("ContributionType", 'CORP', 'Corporation')) \
  .withColumn('ContributerType', regexp_replace("ContributionType", 'EMPO', 'Labor Union')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'FAM', 'Candidate Family')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'IND', 'Individual')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'PCOMZ', 'Party Committee')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'ORG', 'Orgainization')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'LLC', 'Limited Liability Company')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'PCOMP', 'Political Action Committee')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'SPO', ' Candidates Spouse')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'OTHR', 'Other')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'PCOMC', 'Candidate Committee')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'PART', 'Individual')) \


In [ ]:
#Change Payment Method Name
df4=df3.withColumn('PayMethod', regexp_replace('PayMethod','0','Unknown')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','1','Cash')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','2','Check')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','3','Other')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','4','Credit Card')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','5','Money Order')) \

In [ ]:
#Change Borough name
from pyspark.sql.functions import regexp_replace
df5 = df4.withColumn('BoroughName', regexp_replace('BoroughName', 'K', 'Brooklyn')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'M', 'Manhattan')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'Q', 'Queens')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'S', 'Staten Island')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'X', 'Bronx')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'Z', 'Other')) \


In [ ]:
#Call only Mayor (1) and particaptes (P) within Dataframe
df6 = df5.filter((df5.OFFICECD=="1") & (df5.CANCLASS=="P"))
df6.show()

In [ ]:
#Check the above dataframe to make sure only unique values between the OFFICECD is 1 
df6.select('OFFICECD').distinct().show()

In [ ]:
#Drop OFFICECD and CANCLASS
list2 = ['OFFICECD', 'CANCLASS'] 
df7 = df6.drop(*list2)
df7.show()

In [ ]:
# ADD AMNT and MATCHAMNT PrevAmnt
from pyspark.sql.functions import col
df8 = df7.withColumn("TotalAmount", col("Amount")+col("MatchAmt")+col('PrevAmt'))
df8.show()